## Good Code


In [24]:
import numpy as np
import cv2

# Constants: Focal length (F), Width of the object (W) in cm
F = 700  # replace with actual focal length value in pixels
W = 60  # replace with the actual width of the object in cm

# Get the distance (D) of the object from the camera using pixels (P) covered
def calculate_distance(P):
    distance = (F * W) / P
    return distance

# Load the video
video_path = '/content/1075151066-preview.mp4'  # replace with actual path to your video
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error: Video file not accessible.")
    exit()

# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Define the codec and create a VideoWriter object to save the output video
output_path = '/content/output_video_with_distances.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for .mp4 file
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

while True:
    ret, img = cap.read()
    if not ret:
        print("Failed to grab frame or end of video reached")
        break

    # Convert the frame to HSV and apply color detection
    hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # You can adjust these values based on your object color
    lower = np.array([30, 50, 50])  # Example: Yellow color lower bound
    upper = np.array([90, 255, 255])  # Example: Yellow color upper bound
    mask = cv2.inRange(hsv_img, lower, upper)

    # Find contours in the mask
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        if cv2.contourArea(contour) > 1000:  # Ignore small contours
            # Get bounding box for the object
            x, y, w, h = cv2.boundingRect(contour)

            # Calculate pixel width (P)
            P = w

            # Calculate the distance from the camera
            distance = calculate_distance(P)

            # Draw a rectangle around the object
            cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)

            # Display the calculated distance on the video
            cv2.putText(img, f'{distance:.2f} cm', (x, y - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    # Write the processed frame to the output video
    out.write(img)

# Release resources
cap.release()
out.release()

print(f"Video saved at: {output_path}")

Failed to grab frame or end of video reached
Video saved at: /content/output_video_with_distances.mp4


In [27]:
# تحميل YOLOv5 من مستودع GitHub
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -r requirements.txt


Cloning into 'yolov5'...
remote: Enumerating objects: 17067, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 17067 (delta 24), reused 28 (delta 12), pack-reused 17022 (from 1)
Receiving objects: 100% (17067/17067), 15.68 MiB | 21.44 MiB/s, done.
Resolving deltas: 100% (11719/11719), done.
/content/yolov5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 896.3/896.3 kB 16.7 MB/s eta 0:00:00


In [28]:
# تثبيت PyTorch
!pip install torch torchvision torchaudio


In [31]:
# تحميل نموذج مدرب مسبقًا من YOLOv5
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')  # تحميل النموذج المدرب مسبقًا


Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-11-28 Python-3.10.12 torch-2.5.1+cu121 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


## Perfect code

In [32]:
import torch
import cv2
import numpy as np

# Constants: Focal length (F), Width of the object (W) in cm
F = 700  # replace with actual focal length value in pixels
W = 60  # replace with the actual width of the object in cm

# Get the distance (D) of the object from the camera using pixels (P) covered
def calculate_distance(P):
    distance = (F * W) / P
    return distance

# Load YOLOv5 model (pretrained)
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')  # Load YOLOv5s pre-trained model

# Load the video
video_path = '/content/1075151066-preview.mp4'  # replace with actual path to your video
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error: Video file not accessible.")
    exit()

# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Define the codec and create a VideoWriter object to save the output video
output_path = '/content/output_video_with_distances.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for .mp4 file
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

while True:
    ret, img = cap.read()
    if not ret:
        print("Failed to grab frame or end of video reached")
        break

    # Perform object detection using YOLOv5
    results = model(img)  # Perform inference

    # Parse the results to get bounding boxes and class labels
    detections = results.pred[0].cpu().numpy()  # Get predictions (boxes, scores, classes)
    cars = []

    for detection in detections:
        x1, y1, x2, y2, conf, cls = detection
        cls = int(cls)
        if conf > 0.5 and cls == 2:  # Class 2 corresponds to "car" in COCO dataset
            width = int(x2 - x1)
            # Add the car information to the list
            cars.append((x1, y1, width, conf))

    # If cars are detected, choose the closest one
    if cars:
        closest_car = None
        closest_distance = float('inf')

        for car in cars:
            x1, y1, width, conf = car
            P = width  # Using width of the bounding box
            distance = calculate_distance(P)

            if distance < closest_distance:
                closest_car = car
                closest_distance = distance

        # Draw the bounding box and distance for the closest car
        if closest_car:
            x1, y1, width, _ = closest_car
            cv2.rectangle(img, (int(x1), int(y1)), (int(x1 + width), int(y1 + width)), (0, 255, 0), 2)
            cv2.putText(img, f'{closest_distance:.2f} cm', (int(x1), int(y1 - 10)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    # Write the processed frame to the output video
    out.write(img)

# Release resources
cap.release()
out.release()

print(f"Video saved at: {output_path}")


Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-11-28 Python-3.10.12 torch-2.5.1+cu121 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning

Failed to grab frame or end of video reached
Video saved at: /content/output_video_with_distances.mp4


## Edit this to make the barrier allert for the client

In [35]:
!pip install gTTS


In [39]:
from IPython.display import Audio
from gtts import gTTS
import torch
import cv2

# Constants: Focal length (F), Width of the object (W) in cm
F = 700  # replace with actual focal length value in pixels
W = 60  # replace with the actual width of the object in cm

# Get the distance (D) of the object from the camera using pixels (P) covered
def calculate_distance(P):
    distance = (F * W) / P
    return distance

# Load YOLOv5 model (pretrained)
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')  # Load YOLOv5s pre-trained model

# Load the video
video_path = '/content/1075151066-preview.mp4'  # replace with actual path to your video
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error: Video file not accessible.")
    exit()

# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Define the codec and create a VideoWriter object to save the output video
output_path = '/content/output_video_with_alerts.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for .mp4 file
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

played_10m = False  # Track if 10m sound has been played
played_5m = False  # Track if 5m sound has been played

while True:
    ret, img = cap.read()
    if not ret:
        print("End of video reached or failed to grab frame.")
        break

    # Perform object detection using YOLOv5
    results = model(img)  # Perform inference

    # Parse the results to get bounding boxes and class labels
    detections = results.pred[0].cpu().numpy()  # Get predictions (boxes, scores, classes)
    cars = []

    for detection in detections:
        x1, y1, x2, y2, conf, cls = detection
        cls = int(cls)
        if conf > 0.5 and cls == 2:  # Class 2 corresponds to "car" in COCO dataset
            width = int(x2 - x1)
            cars.append((x1, y1, width, conf))

    # If cars are detected, choose the closest one
    if cars:
        closest_car = None
        closest_distance = float('inf')

        for car in cars:
            x1, y1, width, conf = car
            P = width  # Using width of the bounding box
            distance = calculate_distance(P)

            if distance < closest_distance:
                closest_car = car
                closest_distance = distance

        # Draw the bounding box and distance for the closest car
        if closest_car:
            x1, y1, width, _ = closest_car
            cv2.rectangle(img, (int(x1), int(y1)), (int(x1 + width), int(y1 + width)), (0, 255, 0), 2)
            cv2.putText(img, f'{closest_distance:.2f} cm', (int(x1), int(y1 - 10)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

            # Handle sound alerts
            if closest_distance <= 1000 and not played_10m:  # 10 meters or less
                tts = gTTS("There is a car less than 10 meters away!", lang="en")
                tts.save("/content/alert_10m.mp3")
                display(Audio("/content/alert_10m.mp3", autoplay=True))
                played_10m = True
            elif closest_distance <= 500 and not played_5m:  # 5 meters or less
                tts = gTTS("Caution! Car less than 5 meters away!", lang="en")
                tts.save("/content/alert_5m.mp3")
                display(Audio("/content/alert_5m.mp3", autoplay=True))
                played_5m = True
            elif closest_distance > 1000:  # Reset flags if the car moves away
                played_10m = False
                played_5m = False

    # Write the processed frame to the output video
    out.write(img)

# Release resources
cap.release()
out.release()

print(f"Video saved at: {output_path}")


Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-11-28 Python-3.10.12 torch-2.5.1+cu121 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
  with amp.autocast(autocast):



  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):



  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):



  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):



  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with a

...

  with amp.autocast(autocast):



  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):



  with amp.autocast(autocast):



  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with amp.autocast(autocast):

  with a

End of video reached or failed to grab frame.
Video saved at: /content/output_video_with_alerts.mp4
